# House Prices - EDA and models comparison

*Autores: David Tejero Ruiz & Miguel Gil Castilla*

El [dataset](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data) ha sido obtenido de la página web de Kaggle en el apartado de competiciones. Este ha sido escogido debido a que está recomendado como un dataset de "juguete" con el que continuar el aprendizaje en python y ciencia del dato a un nivel más o menos básico. Nuestra idea es realizar un exploratory data análisis aplicando técnicas vistas en clase y ampliando estas con algunas ideas captadas de la red, tras esto trataremos nuestro dataset ya modificado para crear un modelo de regresión con el que predecir la variable respuesta *PriceSale*.

Una particularidad de este dataset es su elevado número de atributos (79), lo que lo hace muy adecuado para poner en práctica algunas técnicas vistas de ingeniería de características. Así, podemos extraer información relevante de ellas para realizar una predicción más o menos precisa del precio de venta de las diferentes viviendas.


Dado que en el conjunto test.csv no se proporciona el valor de la variable respuesta (era objetivo obtenerlo para la competición de Kaggle al que pertenece), nos hemos centrado en el conjunto *train.csv*, que hemos renombrado al archivo *data.csv* que se encuentra en este directorio, y será el conjunto de datos que analicemos.

In [ ]:
import numpy as np

# Importamos el conjunto de datos
import pandas as pd
House_prices = pd.read_csv('data.csv')

# Para evitar avisos innecesarios (FutureWarnings):
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

Notar que Pandas por defecto, trata los valores NA (Not Available) como NaN (Not a Number). Si se analiza la descripción de los datos, NA es un valor que pueden tomar algunos atributos categóricos (no todos), por lo que no se trata de un valor perdido como tal, sino que  indica que no se puede calcular el valor del atributo, porque la vivienda no dispone de el item que se está evaluando. Por ejemplo, en el caso del atributo de calidad de la piscina, NA indica que la vivienda no dispone de piscina, lo cuál es información relevante, y no un valor perdido que tendríamos que imputar.

Hemos analizado la descripción de los diferentes atributos categóricos y aquellos en los que un NA podría aportar información relevante, son:

- Alley

- BsmtQual

- BsmtCond

- BsmtExposure

- BsmtFinType1

- BsmtFinType2

- FireplaceQu

- GarageType

- GarageYrBlt

- GarageFinish

- GarageQual

- GarageCond

- PoolQC

- Fence

- MiscFeature


In [ ]:
na_attributes = ['Alley', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
                 'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageQual', 'GarageCond',
                 'PoolQC', 'Fence', 'MiscFeature']

In [ ]:
# Vemos la cabecera de los datos
House_prices.head()

Como vemos, el dataset está compuesto por 81 columnas, de las cuales 79 serán las características que usaremos en la predicción, y las dos restantes sol el id (que no aporta información), y la variable que queremos predecir (precio de venta de la casa).

Además notamos que existe una riqueza de características alta, encontrando tanto valores numéricos como categóricos, cosa que analizaremos a continuación en el análisis.

In [ ]:
# Vamos a definir un problema de regresión sobre el precio de las casas, por lo que la variable 
# objetivo del dataset será 'SalePrice' (última columna de nuestro dataset como es habitual)
y_data = House_prices['SalePrice']

# Preprocesado de datos

### Análisis de missing values

Como comentamos al inicio, el dataset contiene valores NA, y en algunos casos realmente podrían aportar información útil al decir que no se tiene el objeto que tratamos de evaluar en el atributo. Vamos a ver qué atributos contienen valores NA, y cuántos de estos valores NA contienen.

In [ ]:
# Vamos a contar las apariciones de NaNs en nuestro dataset al completo
print("Hay ", House_prices.isnull().sum().sum(), " valores nulos")

In [ ]:
# Separamos el conjunto de datos en atributos con posibles valores NA y 
# atributos con posibles missing values
House_prices_na = House_prices[na_attributes]
House_prices_mv = House_prices.drop(na_attributes, axis=1)

na_counts = House_prices_mv.isnull().sum()
na_counts = na_counts[na_counts > 0]
print("Atributos que contienen missing values: \n{}".format(na_counts))
print("Hay ", len(na_counts), " atributos que contienen missing values")


In [ ]:
# De entre aquellos en los que NA no es un valor posible del atributo,
# y las apariciones de NaN son datos pérdidos, vamos a ver cuáles son 
# numéricos y cuáles no
House_prices_mv_numerics = House_prices_mv[na_counts.keys()].select_dtypes(include=[np.number])
House_prices_mv_categorics = House_prices_mv[na_counts.keys()].select_dtypes(exclude=[np.number])
print("Atributos numericos que contienen missing values: \n{}".format(House_prices_mv_numerics.keys().to_list()))
print("Atributos categóricos que contienen missing values: \n{}".format(House_prices_mv_categorics.keys().to_list()))


In [ ]:
# Los atributos numéricos que contienen missing values los vamos a rellenar con la media
# de los valores de la columna
House_prices_mv_numerics = House_prices_mv_numerics.fillna(House_prices_mv_numerics.mean())

# Los atributos categóricos que contienen missing values los vamos a rellenar con el valor
# más frecuente de la columna
House_prices_mv_categorics = House_prices_mv_categorics.fillna(House_prices_mv_categorics.mode().iloc[0])

# Vamos a comprobar que ya no hay valores nulos en nuestro dataset
if House_prices_mv_numerics.isnull().sum().sum() == 0 and House_prices_mv_categorics.isnull().sum().sum() == 0:
    print("No hay valores nulos en estos conjuntos")

# Unimos los dos conjuntos de datos como salida de la imputación de valores perdidos
House_prices_mv = pd.concat([House_prices_mv_numerics, House_prices_mv_categorics], axis=1)

Para los valores cuyos NaNs indican un posible valor categorico, vamos a sustituirlos por la cadena "None", para que no se traten como valores perdidos.

In [ ]:
House_prices_na_numerics = House_prices_na.select_dtypes(include=[np.number])
House_prices_na_categorics = House_prices_na.select_dtypes(exclude=[np.number])

print("Atributos numericos que contienen NA: \n{}".format(House_prices_na_numerics.keys().to_list()))
print("Atributos categóricos que contienen NA: \n{}".format(House_prices_na_categorics.keys().to_list()))

In [ ]:
# Mostramos a continuación una gráfica la información anterior
import matplotlib.pyplot as plt

ordenados_na = na_counts.sort_values(ascending=False)
plt.bar(ordenados_na.index, ordenados_na)
plt.xticks(rotation='vertical')
plt.show()

In [ ]:
# Pintamos histograma de la variable objetivo, para ver su distribución
plt.hist(y_data, bins=50)
plt.show()

In [ ]:
# Extraemos estadísticas de la variable respuesta (SalePrice)
y_data.describe()


In [ ]:
# Mostramos los valores de PoolQC respecto al precio de la casa
# plot = House_prices.plot.scatter(x='SalePrice', y=ordenados_na.index[0])
# plt.show()

fig, ax = plt.subplots(2, 2, figsize=(15, 7))
ax[0,0].scatter(House_prices['SalePrice'], House_prices[ordenados_na.index[0]])
ax[0,0].set_xlabel('SalePrice')
ax[0,0].set_ylabel(ordenados_na.index[0])
ax[0,0].grid()

ax[0,1].scatter(House_prices['SalePrice'], House_prices[ordenados_na.index[1]])
ax[0,1].set_xlabel('SalePrice')
ax[0,1].set_ylabel(ordenados_na.index[1])
ax[0,1].grid()

ax[1,0].scatter(House_prices['SalePrice'], House_prices[ordenados_na.index[2]])
ax[1,0].set_xlabel('SalePrice')
ax[1,0].set_ylabel(ordenados_na.index[2])
ax[1,0].grid()

ax[1,1].scatter(House_prices['SalePrice'], House_prices[ordenados_na.index[3]])
ax[1,1].set_xlabel('SalePrice')
ax[1,1].set_ylabel(ordenados_na.index[3])
ax[1,1].grid()

plt.show()

Como se puede observar en las gráficas comparativas, vemos que los valores que no son NA de los atributos categóricos se mantienen cerca de la media de la variable respuesta con lo cual no aportan información relevante. Es por esto por lo que decidimos directamente eliminar estos atributos categóricos.

In [ ]:
# Eliminamos los 4 atributos que más NA contienen 
categorical_atr.remove('PoolQC')
categorical_atr.remove('MiscFeature')
categorical_atr.remove('Alley')
categorical_atr.remove('Fence')

In [ ]:


# Para tratar los diferentes atributos lo primero es distinguir entre las variables cuantitativas 
# (numéricas) y cualitativas (categóricas)

# Ya que el tipo de dato 'object' engloba a las variables categóricas y a las variables de tipo
# string, se puede usar dicho tipo de dato para distinguir las variables en numéricas y categóricas
numerical_atr = [col for col in House_prices.columns if House_prices.dtypes[col] != 'object']
categorical_atr = [col for col in House_prices.columns if House_prices.dtypes[col] == 'object']

# Eliminamos las variables 'SalePrice' (variable objetico) e 'Id' (no aporta información) de la
# lista de variables numéricas
numerical_atr.remove('SalePrice')
numerical_atr.remove('Id')

print("Hay ", len(numerical_atr), " datos numéricos: ",numerical_atr)
print("Hay ", len(categorical_atr), " datos categóricos: ",categorical_atr)

### Construcción de nuestro dataset
Tras haber realizado un primer análisis inicial, vamos a construir un primer dataset de partida, que será el que usaremos para realizar el análisis exploratorio de datos y la construcción de los modelos de predicción.

In [ ]:
# Creamos nuestro dataset 
numerical_data = House_prices[numerical_atr]
categorical_data = House_prices[categorical_atr]

# Concatenamos las variables numéricas y las variables categóricas para obtener el dataset final
X_data = pd.concat([numerical_data, categorical_data], axis=1)

In [ ]:
# Dividimos el conjunto de datos en tres subconjuntos: entrenamiento, validación y test
# En una proprorción 50% - 20% - 30% respectivamente

train_size = 0.5
val_size = 0.2
test_size = 0.3

from sklearn.model_selection import train_test_split
X_train, X_aux, y_train, y_aux = train_test_split(X_data, y_data, test_size=(1.0-train_size), random_state=41)
X_val, X_tes, y_val, y_test = train_test_split(X_aux, y_aux, test_size=test_size/(val_size+test_size), random_state=41)


In [ ]:
# Normalizamos las variables numéricas para que todas tengan media 0 y desviación típica 1
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train[numerical_atr])
X_train[numerical_atr] = scaler.transform(X_train[numerical_atr])
X_val[numerical_atr] = scaler.transform(X_val[numerical_atr])
X_tes[numerical_atr] = scaler.transform(X_tes[numerical_atr])

In [ ]:
# Mostramos las variables categoricas
X_train[categorical_atr].head()

## Multicolinealidad

In [ ]:
# Coeficiente de correlación de Pearson entre los atributos
import seaborn as sns
plt.figure(figsize=(15,15))
plt.title('Coeficiente de Correlación de Pearson entre los atributos', y=1.05, size=15)

sns.heatmap(House_prices.corr(),linewidths=0.1,vmax=1.0, 
            square=True, cmap='viridis', linecolor='white', annot=True)

plt.xticks(rotation=90)
plt.yticks(rotation=0)
plt.show()

# Modelos

In [ ]:
# Dividimos los datos en entrenamiento y prueba
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, random_state=0)



COSAS QUE HAY QUE HACER:

**Preprocesado de datos**
- Eliminar columnas con muchos valores perdidos
- One-hot encoding / Label encoding (categorical variables)
- Normalizar datos: StandardScaler, MinMaxScaler, RobustScaler 
- Eliminar outliers: IQR, Z-score, etc.  (?)
- Discretizado de variables continuas (?): Binning, etc.


**Ingeneering features**
- Reducción de dimensionalidad: *PCA*, LDA, etc.
- Selección de variables - Filter Methods: Correlation, Chi2, ANOVA, etc.
- Multicolinealidad: VIF, etc. 
- Crear nuevas variables.

**Posibles Modelos**
- Regresión lineal
- Regresión polinómica
- Regresión Ridge
- Regresión Lasso
- Regresión ElasticNet
- SVR
- Random Forest
- **Gradient Boosting**
- XGBoost
- LightGBM
- CatBoost

**Evaluación de modelos**
- MSE
- k-fold cross validation
- Recall, Precision, F1-score, etc. (?)